# Melt basics

In [27]:
import MagmaPandas as mp

Import some melt data from a csv file

In [28]:
melt_file = "./data/melts.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
melt.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,
PI032-04-01,49.402603,14.539062,6.509044,10.058735,9.757536,3.737374,0.697769,0.132497,2.548581,0.289823,0.628502,1.440208,0.078857,0.145452,0.033958,100
PI032-04-02,48.832207,15.116896,6.508618,9.688642,9.927146,3.564377,0.940308,0.135905,2.652552,0.336327,0.672345,1.330865,0.083879,0.165394,0.044540,100
PI041-02-02,49.124916,16.950138,4.861451,9.183852,10.238358,3.788617,1.077791,0.153258,2.804343,0.561031,0.463920,0.655494,0.047654,0.068020,0.021157,100
PI041-03-01,45.941086,15.753453,6.909844,10.996510,10.778833,3.346468,1.144852,0.112117,3.153303,0.551992,0.797715,0.293341,0.079499,0.086863,0.054123,99
PI041-03-03,45.313339,15.909531,6.831676,11.175276,10.966834,3.417548,1.167550,0.088899,3.262463,0.517891,0.821116,0.311131,0.081713,0.079495,0.055540,100


On top of the regular MagmaFrame methods, Melt has additional methods for calculating

- NBO/T,
- density,
- viscosity
- liquidus temperatures,
- Fe3+/Fe2+ ratios,
- melt FeO and Fe2O3,
- olivine-melt Fe-Mg partitioning coefficients and
- H2O-CO2 saturation pressures.

Let's check the current configuration of MagmaPandas ...

In [29]:
print(mp.configuration)


################## MagmaPandas ###################
##################################################
General settings__________________________________
fO2 buffer.....................................QFM
ΔfO2.............................................0
Melt Fe3+/Fe2+.........................borisov2018
Kd Fe-Mg ol-melt........................toplis2005
Melt thermometer....................putirka2008_16
Volatile solubility model.......iaconomarziano2012
Volatile species.............................mixed
##################################################



... and calculate liquidus temperatures with a melt-only thermometer at 4 kbar. The configuration is currently set to use equation 15 from Putirka (2008).

In [30]:
P_bar = 4e3

temperatures = melt.temperature(P_bar=P_bar)
temperatures

name
PI032-04-01    1441.689112
PI032-04-02    1438.624308
PI041-02-02    1395.139632
PI041-03-01    1442.484997
PI041-03-03    1441.510214
PI041-05-04    1370.343045
PI041-05-06    1399.939176
PI041-07-01    1440.598613
PI041-07-02    1439.236321
PI052-01-02    1368.655861
Name: T_K, dtype: float64

We can switch to a different thermometer by changing the configuration:

In [31]:
mp.configuration.melt_thermometer = "putirka2008_16"

temperatures_new = melt.temperature(P_bar=P_bar)
temperatures_new

name
PI032-04-01    1441.689112
PI032-04-02    1438.624308
PI041-02-02    1395.139632
PI041-03-01    1442.484997
PI041-03-03    1441.510214
PI041-05-04    1370.343045
PI041-05-06    1399.939176
PI041-07-01    1440.598613
PI041-07-02    1439.236321
PI052-01-02    1368.655861
Name: T_K, dtype: float64

With these temperatures we can calculate melt Fe3+/Fe2+ ratios. The configuration is currently set to fO2 buffered at QFM+1 and the Borisov et al. (2018) Fe3+/Fe2+ model.

In [32]:
melt.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,
PI032-04-01,49.402603,14.539062,6.509044,10.058735,9.757536,3.737374,0.697769,0.132497,2.548581,0.289823,0.628502,1.440208,0.078857,0.145452,0.033958,100
PI032-04-02,48.832207,15.116896,6.508618,9.688642,9.927146,3.564377,0.940308,0.135905,2.652552,0.336327,0.672345,1.330865,0.083879,0.165394,0.044540,100
PI041-02-02,49.124916,16.950138,4.861451,9.183852,10.238358,3.788617,1.077791,0.153258,2.804343,0.561031,0.463920,0.655494,0.047654,0.068020,0.021157,100
PI041-03-01,45.941086,15.753453,6.909844,10.996510,10.778833,3.346468,1.144852,0.112117,3.153303,0.551992,0.797715,0.293341,0.079499,0.086863,0.054123,99
PI041-03-03,45.313339,15.909531,6.831676,11.175276,10.966834,3.417548,1.167550,0.088899,3.262463,0.517891,0.821116,0.311131,0.081713,0.079495,0.055540,100


In [33]:
print(mp.configuration)


################## MagmaPandas ###################
##################################################
General settings__________________________________
fO2 buffer.....................................QFM
ΔfO2.............................................0
Melt Fe3+/Fe2+.........................borisov2018
Kd Fe-Mg ol-melt........................toplis2005
Melt thermometer....................putirka2008_16
Volatile solubility model.......iaconomarziano2012
Volatile species.............................mixed
##################################################



In [34]:
moles = melt.moles()

In [35]:
moles.FeO_Fe2O3_calc(0.33, wtpc=False)

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,CO2,H2O,F,S,Cl,total,Fe2O3
name,,,,,,,,,,,,,,,,,
PI032-04-01,0.498648,0.086478,0.097943,0.108783,0.061930,0.036570,0.004492,0.001133,0.019353,0.001238,0.008661,0.048484,0.002517,0.002751,0.000581,0.989782,0.010218
PI032-04-02,0.496051,0.090491,0.098564,0.105452,0.063410,0.035101,0.006093,0.001169,0.020272,0.001446,0.009325,0.045090,0.002695,0.003148,0.000767,0.989537,0.010463
PI041-02-02,0.518555,0.105437,0.076501,0.103871,0.067958,0.038769,0.007257,0.001370,0.022270,0.002507,0.006686,0.023077,0.001591,0.001345,0.000378,0.988787,0.011213
PI041-03-01,0.480702,0.097135,0.107784,0.123283,0.070919,0.033945,0.007641,0.000994,0.024822,0.002445,0.011396,0.010237,0.002631,0.001703,0.000960,0.988298,0.011702
PI041-03-03,0.474557,0.098185,0.106660,0.125399,0.072220,0.034697,0.007800,0.000789,0.025705,0.002296,0.011741,0.010868,0.002706,0.001560,0.000986,0.988084,0.011916
PI041-05-04,0.512278,0.117337,0.061949,0.107859,0.064936,0.047763,0.010978,0.001253,0.020040,0.003742,0.007460,0.016494,0.002973,0.002438,0.001071,0.989286,0.010714
PI041-05-06,0.492420,0.106249,0.075050,0.101574,0.077597,0.041217,0.009602,0.001508,0.029047,0.002786,0.009285,0.020756,0.003609,0.002551,0.001141,0.987197,0.012803
PI041-07-01,0.473225,0.094020,0.103617,0.108615,0.074906,0.031789,0.008501,0.001267,0.027836,0.002460,0.006689,0.035661,0.002488,0.003207,0.001000,0.987641,0.012359
PI041-07-02,0.479135,0.096393,0.103903,0.113359,0.075639,0.032479,0.009113,0.001195,0.027930,0.002739,0.005337,0.021588,0.002444,0.002763,0.001021,0.987519,0.012481


In [36]:
mp.configuration.Fe3Fe2_model = "borisov2018"
Fe3Fe2_QFM1 = melt.Fe3Fe2(T_K=temperatures, P_bar=P_bar)
Fe3Fe2_QFM1

name
PI032-04-01    0.192937
PI032-04-02    0.190789
PI041-02-02    0.173552
PI041-03-01    0.192178
PI041-03-03    0.196175
PI041-05-04    0.181691
PI041-05-06    0.187878
PI041-07-01    0.195070
PI041-07-02    0.191716
PI052-01-02    0.195356
dtype: float64

Changing the fO2 buffer to lower log offset values reduces the amount of Fe3+

In [37]:
mp.configuration.dfO2 = 0

melt.Fe3Fe2(T_K=temperatures, P_bar=P_bar)

name
PI032-04-01    0.192937
PI032-04-02    0.190789
PI041-02-02    0.173552
PI041-03-01    0.192178
PI041-03-03    0.196175
PI041-05-04    0.181691
PI041-05-06    0.187878
PI041-07-01    0.195070
PI041-07-02    0.191716
PI052-01-02    0.195356
dtype: float64

With our Fe3+/Fe2+ ratios we can calculate melt FeO and Fe2O3 contents

In [38]:
melt.FeO_Fe2O3_calc(Fe3Fe2=Fe3Fe2_QFM1)

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,CO2,H2O,F,S,Cl,Fe2O3,total
name,,,,,,,,,,,,,,,,,
PI032-04-01,49.315947,14.513559,6.497627,10.041091,8.165077,3.730818,0.696545,0.132265,2.544111,0.289315,0.627400,1.437682,0.078719,0.145197,0.033898,1.750750,100.0
PI032-04-02,48.745878,15.090171,6.497111,9.671513,8.321875,3.558075,0.938646,0.135665,2.647862,0.335733,0.671157,1.328513,0.083731,0.165102,0.044461,1.764507,100.0
PI041-02-02,49.042236,16.921610,4.853269,9.168395,8.709566,3.782241,1.075977,0.153000,2.799624,0.560087,0.463139,0.654390,0.047573,0.067906,0.021122,1.679865,100.0
PI041-03-01,45.852377,15.723034,6.896501,10.975276,9.023837,3.340007,1.142641,0.111900,3.147214,0.550926,0.796175,0.292775,0.079345,0.086695,0.054018,1.927276,100.0
PI041-03-03,45.222774,15.877733,6.818022,11.152940,9.149930,3.410718,1.165216,0.088721,3.255942,0.516856,0.819475,0.310509,0.081549,0.079336,0.055429,1.994848,100.0
PI041-05-04,47.637183,18.516223,3.864216,9.361043,8.126579,4.581581,1.600461,0.137524,2.477081,0.822031,0.508098,0.459869,0.087420,0.120972,0.058793,1.640927,100.0
PI041-05-06,46.272040,16.942774,4.730699,8.908262,9.761997,3.995287,1.414517,0.167331,3.628101,0.618478,0.639027,0.584787,0.107242,0.127910,0.063273,2.038276,100.0
PI041-07-01,45.764297,15.429679,6.721680,9.803361,9.639725,3.171182,1.288853,0.144685,3.578237,0.562046,0.473776,1.034022,0.076072,0.165536,0.057054,2.089796,100.0
PI041-07-02,45.776560,15.628050,6.658930,10.108046,9.643705,3.200864,1.364980,0.134766,3.546947,0.618237,0.373498,0.618406,0.073834,0.140891,0.057567,2.054718,100.0


Our melts also contain H2O and CO2, so let's calculate volatile saturation pressures. In the configuration we see that we're currently using the Iacono-Marziano et al. (2012) model with mixed CO2-H2O fluids:

In [39]:
print(mp.configuration)


################## MagmaPandas ###################
##################################################
General settings__________________________________
fO2 buffer.....................................QFM
ΔfO2.............................................0
Melt Fe3+/Fe2+.........................borisov2018
Kd Fe-Mg ol-melt........................toplis2005
Melt thermometer....................putirka2008_16
Volatile solubility model.......iaconomarziano2012
Volatile species.............................mixed
##################################################



In [40]:
pressures_IM2012 = melt.volatile_saturation_pressure(T_K=temperatures)
pressures_IM2012

Saturation pressure...    |█████████████████████████| 100% [10/10] in 2.5s (3.17


name
PI032-04-01    5853.465872
PI032-04-02    6504.086975
PI041-02-02    4772.936412
PI041-03-01    6417.666636
PI041-03-03    6296.956179
PI041-05-04    4100.520755
PI041-05-06    5150.179572
PI041-07-01    4776.667586
PI041-07-02    3804.242787
PI052-01-02    2586.324019
dtype: float64

Lets change the volatile solubility model to Allison et al. (2022):

In [41]:
mp.configuration.volatile_solubility = 'allison2022'
print(mp.configuration)


################## MagmaPandas ###################
##################################################
General settings__________________________________
fO2 buffer.....................................QFM
ΔfO2.............................................0
Melt Fe3+/Fe2+.........................borisov2018
Kd Fe-Mg ol-melt........................toplis2005
Melt thermometer....................putirka2008_16
Volatile solubility model..............allison2022
Volatile species.............................mixed
##################################################



In [42]:
pressures_A22 = melt.volatile_saturation_pressure(T_K=temperatures)
pressures_A22

Saturation pressure...    |█████████████████████████| 100% [10/10] in 1.1s (8.67


name
PI032-04-01    4759.038948
PI032-04-02    5370.482770
PI041-02-02    4878.423290
PI041-03-01    5792.580142
PI041-03-03    5976.245167
PI041-05-04    5231.131109
PI041-05-06    5797.897481
PI041-07-01    4503.986268
PI041-07-02    3904.017937
PI052-01-02    2594.307474
dtype: float64